<a href="https://colab.research.google.com/github/jmestradag/thefts_and_arrests_Colombia/blob/main/Thefts_arrests_analysis_Colombia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Descriptive exploratory data analysis for thefts and arrests data in Colombia from 2010 to 2020



## Introduction

Crime with most impact in Colombia is theft, as per National Police database, being the most frequent crime, it helps to explain security perception within citizens and general population. Looking into social media, it looks like that there is a generallized concern about this specific crime now during reactivation period after 5 months of permanent quarantine and lockdown due to coronavirus pandemic disease. 

This both facts explains why this descriptive analysis is focused on this crime instead of others that might be more outrageous and, of course, because there is more data publicly available to practice data science skills with it.

This project tries to analyse official data in order to either reject or confirm if citizens perception is based on data reported by themselves to National Police.

### Questions for project:

1. Which areas (states/cities) are top 5 most impacted by thefts? Is there any coincidence with areas having most unsatisfied basic needs or poverty levels? Or is inequality in main cities affecting thefts levels?

2. How we can compare thefts levels on these areas based on differences in their population distribution? Is there a metric for comparison against foreign cities around the world?

3. Is there a trend in data that can be used to answer a deteriorating security perception within citizens?

4. How efficiently have performed National Police during analysis period to answer security perception of citizens regarding thefts?

5. How is recidivism and overcrowding levels in Colombia's prisons are affecting security perception on citizens?

#### Author: jmestradag@gmail.com

### Data Sources:

[Crime Database - National Police](
https://www.policia.gov.co/grupo-informaci%C3%B3n-criminalidad/estadistica-delictiva)

[Population distribution for Colombia - DANE](https://www.dane.gov.co/files/censo2018/informacion-tecnica/CNPV-2018-Poblacion-Ajustada-por-Cobertura.xls)

[Unsatisfied Basic Needs data in Colombia - DANE](https://www.dane.gov.co/files/censo2018/informacion-tecnica/CNPV-2018-NBI.xlsx)

[Prison population dashboard - INPEC](http://200.91.226.18:8080/jasperserver-pro/flow.html?_flowId=dashboardRuntimeFlow&dashboardResource=/public/DEV/dashboards/Dash__Poblacion_Intramural&j_username=inpec_user&j_password=inpec)

[Stats dashboards - INPEC](https://www.inpec.gov.co/estadisticas-/tableros-estadisticos)



<div class="alert alert-block alert-info" style="margin-top: 20px">
         <img src="https://upload.wikimedia.org/wikipedia/commons/a/a1/Escudo_Polic%C3%ADa_Nacional_de_Colombia.jpg" width="250" align="center">
    </a>
        <img src="https://seeklogo.com/images/C/colombia-dane-logo-CEC894F9A8-seeklogo.com.png" width="250" align="center">
    </a>
    <img src="https://www.inpec.gov.co/documents/20143/46642/3110245.PNG/751d7e05-ee7e-b529-0f64-1b934ea9e114?t=1511790311995" width="250" align="center">
    </a>

</div>


DANE is the institution in charge of handling statistics in Colombia.

INPEC is the institution in charge of managing prisons and inmates in Colombia.

In [1]:
#This is just for Colab on drive ....
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
# Disable warnings in Anaconda
import warnings
warnings.filterwarnings('ignore')

#Libraries for analysis
import pandas as pd
import numpy as np

#Plotting libraries
import plotly.express as px
import plotly.graph_objs as go

## Data capture

### Theft Data

In [101]:
#Create a new dataframe for reading thefts details from 2010 - 2020 upto Nov 30th in data source from police
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/hurtos_a_personas_2010_2020.csv')
#df

In [4]:
#Gather population for states in order to get rates and properly compare them instead of using raw data
pop_states = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/CNPV-2018-Poblacion-Ajustada-por-Cobertura.xlsm',
                           sheet_name='Ajuste por Cobertura CNPV 2018')
#data cleaning
pop_states.drop(columns=['Código DIVIPOLA', 'CABECERA', 'CENTROS POBLADOS Y RURAL DISPERSO'], inplace=True)
pop_states.rename(columns={'NOMBRE DEPARTAMENTO':'DEPARTAMENTO', 'TOTAL':'TOTAL'}, inplace=True)
pop_states.DEPARTAMENTO = pop_states.DEPARTAMENTO.str.upper()
pop_states.sort_values(by='TOTAL', ascending=False, inplace=True)
pop_states.head()

,DEPARTAMENTO,TOTAL
2,CUNDINAMARCA,10331626
0,ANTIOQUIA,6407102
22,VALLE,4475886
1,ATLÁNTICO,2535517
19,SANTANDER,2184837


In [109]:
#Gather population for cities in order to get rates and properly compare them instead of using raw data
pop_cities = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/CNPV-2018-Poblacion-Ajustada-por-Cobertura.xlsm',
                           sheet_name='Ajuste por Cobertura CNPV Mpios')
#data cleaning
pop_cities.drop(columns=['Código DIVIPOLA', 'CABECERA', 'CENTROS POBLADOS Y RURAL DISPERSO'], inplace=True)
pop_cities.rename(columns={'NOMBRE DEPARTAMENTO':'DEPARTAMENTO', 'NOMBRE MUNICIPIO':'MUNICIPIO','TOTAL':'TOTAL'}, inplace=True)
pop_cities.DEPARTAMENTO = pop_cities.DEPARTAMENTO.str.upper()
pop_cities.MUNICIPIO = pop_cities.MUNICIPIO.str.upper()
pop_cities.sort_values(by='TOTAL', ascending=False, inplace=True)
pop_cities.head(12)

,DEPARTAMENTO,MUNICIPIO,TOTAL
148,CUNDINAMARCA,BOGOTÁ D.C.,7412566
0,ANTIOQUIA,MEDELLÍN,2427129
1005,VALLE DEL CAUCA,CALI,2227642
125,ATLÁNTICO,BARRANQUILLA,1206319
149,BOLÍVAR,CARTAGENA,973045
779,NORTE DE SANTANDER,CÚCUTA,711715
544,CUNDINAMARCA,SOACHA,660179
144,ATLÁNTICO,SOLEDAD,603999
845,SANTANDER,BUCARAMANGA,581130
686,META,VILLAVICENCIO,531275


The most populated states are driven by its main capital cities as can be seen on last 2 dataframes. 

Most population distribution is focused on cities (urban) instead of towns or counties (rural). 

Top 5 most populated cities in Colombia are:

1. Bogota
2. Medellin
3. Cali
4. Barranquilla
5. Cartagena

These data is based on 2018 census from DANE which is the latest official stat about population.

In [102]:
#Gather population for states in order to get rates and properly compare them instead of using raw data
ubn_states = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/CNPV-2018-NBI.xlsx',
                           sheet_name='Departamento')
#ubn_states.head()

In [103]:
#Gather population for cities in order to get rates and properly compare them instead of using raw data
ubn_cities = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/CNPV-2018-NBI.xlsx',
                           sheet_name='Municipios')
#ubn_cities.head()

### Arrests data

In [8]:
#Create a new dataframe for reading arrests details from 2011 - 2010 not included in original data source from police
df2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/capturas_2011.xlsx',
                   skiprows=10, nrows=127624)
#df2

In [9]:
#Create a new dataframe for reading arrests details from 2012
df3 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/capturas_2012.xlsx',
                   skiprows=10, nrows=143860)
#df3

In [10]:
#Create a new dataframe for reading arrests details from 2013
df4 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/capturas_2013.xlsx',
                   skiprows=10, nrows=151160)
#df4

In [11]:
#Create a new dataframe for reading arrests details from 2014
df5 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/capturas_2014.xlsx',
                   skiprows=10, nrows=149313)
#df5

In [12]:
#Create a new dataframe for reading arrests details from 2015
df6 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/capturas_2015.xlsx',
                   skiprows=10, nrows=148721)
#df6

In [13]:
#Create a new dataframe for reading arrests details from 2016
df7 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/capturas_2016.xlsx',
                   skiprows=10, nrows=143952)
#df7

In [14]:
#Create a new dataframe for reading arrests details from 2017
df8 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/capturas_2017.xlsx',
                   skiprows=10, nrows=143418)
#df8

In [15]:
#Create a new dataframe for reading arrests details from 2018
df9 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/capturas_2018.xlsx',
                   skiprows=10, nrows=139293)
#df9

In [16]:
#Create a new dataframe for reading arrests details from 2019
df10 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/capturas_2019.xlsx',
                   skiprows=10, nrows=232682)
#df10

In [17]:
#Create a new dataframe for reading arrests details from 2020 - upto Nov 30th
df11 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/capturas_2020.xlsx',
                   skiprows=9, nrows=99475)
#df11

In [18]:
#Create one single dataframe with arrests details from 2011 through 2020. ---> 2010 not included in National Police Database ***
#dataframes list
data_frames = [df2, df3, df4, df5, df6, df7, df8, df9, df10, df11]

# Define column names
colNames = ('DEPARTAMENTO',	'MUNICIPIO', 'CODIGO DANE',	'FECHA', 'GENERO', 'DELITO', 'EDAD', 'CANTIDAD')

# Define a dataframe with the required column names
arrests = pd.DataFrame(columns = colNames)

#for loop to get one dataframe
for dfx in data_frames:
    arrests = arrests.append(dfx)

arrests

,DEPARTAMENTO,MUNICIPIO,CODIGO DANE,FECHA,GENERO,DELITO,EDAD,CANTIDAD
0,AMAZONAS,LETICIA (CT),91001000.0,2011-01-01,MASCULINO,ARTÍCULO 189. VIOLACIÓN DE HABITACIÓN AJENA.,ADULTOS,1
1,AMAZONAS,LETICIA (CT),91001000.0,2011-01-01,MASCULINO,ARTÍCULO 111. LESIONES PERSONALES,ADULTOS,1
2,AMAZONAS,LETICIA (CT),91001000.0,2011-01-03,MASCULINO,ARTÍCULO 296. FALSEDAD PERSONAL,ADULTOS,1
3,AMAZONAS,LETICIA (CT),91001000.0,2011-01-04,FEMENINO,ARTÍCULO 111. LESIONES PERSONALES,ADULTOS,1
4,AMAZONAS,LETICIA (CT),91001000.0,2011-01-05,MASCULINO,"ARTÍCULO 376. TRÁFICO, FABRICACIÓN O PORTE DE ...",ADULTOS,1
...,...,...,...,...,...,...,...,...
99470,VICHADA,SANTA ROSALÍA,99624000.0,2020-06-25,MASCULINO,"ARTÍCULO 376. TRÁFICO, FABRICACIÓN O PORTE DE ...",ADULTOS,1
99471,VICHADA,SANTA ROSALÍA,99624000.0,2020-11-07,MASCULINO,ARTÍCULO 429. VIOLENCIA CONTRA SERVIDOR PÚBLICO,ADULTOS,1
99472,VICHADA,SANTA ROSALÍA,99624000.0,2020-11-07,MASCULINO,ARTÍCULO 111. LESIONES PERSONALES,ADULTOS,1
99473,VICHADA,SANTA ROSALÍA,99624000.0,2020-11-11,MASCULINO,ARTÍCULO 429. VIOLENCIA CONTRA SERVIDOR PÚBLICO,ADULTOS,1


This dataframe contains all of the arrests performed by National Police with multiple crimes not just due to thefts. There are almost 1.5 million cases recorded during 2011 - 2020 period.

### Prison data

In [19]:
#Create a new dataframe for reading recidivism totals from 2016 - 2020. Other periods not included in original data source from INPEC
rcdv = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/reincidencia_total.xlsx')
rcdv

,Periodo,Total Intramural,Total Domiciliaria,Total Vigilancia,Reincidencia
0,2016,14737,3530,483,18750
1,2017,15311,4365,506,20182
2,2018,16079,4417,627,21123
3,2019,18077,4646,685,23408
4,2020,16056,5876,890,22822


In [20]:
#Create a new dataframe for detention months of unconvicted inmates from 2017 - 2020. Other periods not included in original data source from INPEC
dmu = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/meses_detencion_sindicados.xlsx')
dmu

,Periodo,0-5,6-10,11-15,16-20,21-25,26-30,31-35,>36
0,2017,12795,7546,4385,2766,1599,1162,918,3593
1,2018,12997,9274,5229,3124,1850,1175,877,3331
2,2019,10881,9656,5580,4319,2083,1597,1027,3284
3,2020,1571,1659,4633,4337,2764,2204,1194,3781


In [21]:
#Create a new dataframe for detention years of convicted inmates from 2017 - 2020. Other periods not included in original data source from INPEC
dyc = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/annos_prision_condenados.xlsx')
dyc

,Periodo,0-5,6-10,11-15,16-20,21-25,26-30,31-35,>36
0,2017,25926,22181,10548,9759,3967,2488,1783,3169
1,2018,26796,21355,10648,10254,4032,2464,1900,3290
2,2019,28520,21676,10732,10701,4110,2464,1984,3440
3,2020,22032,19710,10301,10861,4061,2418,1968,3529


In [22]:
#Create a new dataframe for capacity and total prison population from 2014 - 2020. Other periods not included in original data source from INPEC
tpp = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Projects/hurtos_personas_2010_2020/consolidadoSituacionJuridica.xlsx')
tpp

,Periodo,Capacidad,Poblacion,Sindicados,Condenados,Actualizacion
0,2014,77874,114227,37079,76792,356
1,2015,77953,121318,40944,79732,642
2,2016,78420,119033,37056,81553,424
3,2017,79211,115383,34818,79878,687
4,2018,80227,119172,37927,80620,625
5,2019,80260,122679,38473,83462,744
6,2020,80683,97594,22122,74829,643


## Data Wrangling - Country level

###Theft raw data

These analyses and plots are for raw aggregated data without adjusting for population levels.

In [104]:
#change format to DATE column in order to convert later to datetime format
df["fecha"] = pd.to_datetime(df["FECHA HECHO"]).dt.strftime('%Y/%m/%d')

# To convert a string to a datetime
df["DateTime"] = pd.to_datetime(df["fecha"], format="%Y/%m/%d")

# Set index as DateTime for plotting purposes
df = df.set_index(["DateTime"])

# To get month and year column
df["month"] = pd.DatetimeIndex(df["fecha"]).month
df["year"] = pd.DatetimeIndex(df["fecha"]).year
df['month_year'] = pd.to_datetime(df["fecha"]).dt.to_period('M')
df["day"] = pd.to_datetime(df["fecha"]).dt.weekday
#df.head()

In [60]:
#Creating a pivot table for each year to get a close up on values per year
df_pivot_year = df.groupby(["year"])["CANTIDAD"].sum().reset_index().sort_values(by="year")
px.line(x="year" , y="CANTIDAD", data_frame=df_pivot_year, title="Thefts per Year in Colombia - Jan 01st 2010 upto Nov 30th 2020")

#### Thefts per year finding:

There is an upward trend in thefts at country level since 2010 upto 2019; but there is a change in slope after 2015. 

2020 was an atypical year due to lockdown.

In [25]:
#cleaning of some cells from armas medios column to uniform data values from it for ease of plotting and analysis based on synomyms
df["ARMAS MEDIOS"].replace('ARMA BLANCA / CORTOPUNZANTE', 'ARMAS BLANCAS', inplace=True)  #same category, different name
df["ARMAS MEDIOS"].replace('CORTANTES', 'ARMAS BLANCAS', inplace=True)   #same category, different name
df["ARMAS MEDIOS"].replace('NO REPORTADO', 'SIN EMPLEO DE ARMAS', inplace=True)  #same category, different name
df["ARMAS MEDIOS"].replace('-', 'SIN EMPLEO DE ARMAS', inplace=True)  #same category, different name
df["ARMAS MEDIOS"].replace('CORTOPUNZANTES', 'ARMAS BLANCAS', inplace=True)  #same category, different name
df["ARMAS MEDIOS"].replace('PUNZANTES', 'ARMAS BLANCAS', inplace=True)  #same category, different name
df["ARMAS MEDIOS"].replace('NO REGISTRA', 'SIN EMPLEO DE ARMAS', inplace=True)  #same category, different name
df["ARMAS MEDIOS"].replace('PALANCAS', 'CONTUNDENTES', inplace=True)  #same category, different name
df["ARMAS MEDIOS"].replace('SUSTANCIAS TOXICAS', 'JERINGA', inplace=True)  #same category, different name
df["ARMAS MEDIOS"].replace('DIRECTA', 'LLAVE MAESTRA', inplace=True)  #same category, different name
df["ARMAS MEDIOS"].replace('ESCOPOLAMINA', 'ALUCINOGENOS', inplace=True)  #same category, different name

#Creating a pivot table for each year to get a close up on values per year
df_pivot_weapons = df.groupby(["ARMAS MEDIOS"])["CANTIDAD"].sum().reset_index().sort_values(by="CANTIDAD", ascending=False)
fig = px.bar(df_pivot_weapons, x='ARMAS MEDIOS', y='CANTIDAD', title='Weapons used during robbery in Colombia - Jan 01st 2010 upto Nov 30th 2020')
fig.show()

In [105]:
#df_pivot_weapons.sort_values(by="CANTIDAD", ascending=False)

#### Weapons used finding:

Almost 50% of thefts are made without weapons (non violent thefts), the remaining 50% are violent with risk of harm or life threat.

In [27]:
#cleaning of some cells from armas medios column to uniform data values from it for ease of plotting and analysis based on synomyms
df["GENERO"].replace('NO REGISTRA', 'NO REPORTA', inplace=True)  #same category, different name
df["GENERO"].replace('NO REPORTADO', 'NO REPORTA', inplace=True)   #same category, different name
#Creating a pivot table for each year to get a close up on values per year
df_pivot_gender = df.groupby(["GENERO"])["CANTIDAD"].sum().reset_index().sort_values(by="GENERO")
fig = px.bar(df_pivot_gender, x='GENERO', y='CANTIDAD', title='Gender most impacted by robbery in Colombia - Jan 01st 2010 upto Nov 30th 2020')
fig.show()

In [106]:
#df_pivot_gender.sort_values(by="CANTIDAD", ascending=False)

#### Gender assaulted finding:

60% of thefts are made to males and 40% to females. It is curious, since most common belief is that females are more prone to be assaulted in robbery.

In [29]:
#checking which 'departamentos' (states) are more prone to theft and robbery
df_pivot_states = df.groupby(["DEPARTAMENTO"])["CANTIDAD"].sum().reset_index().sort_values(by="CANTIDAD", ascending=False)
df_pivot_states.head(10)

,DEPARTAMENTO,CANTIDAD
12,CUNDINAMARCA,621378
1,ANTIOQUIA,174183
29,VALLE,168055
3,ATLÁNTICO,85586
26,SANTANDER,80744
28,TOLIMA,44181
19,META,40924
20,NARIÑO,39009
4,BOLÍVAR,34714
17,HUILA,34283


In [30]:
#Graph for top 5 of states in Colombia most impacted by thefts
fig = px.bar(df_pivot_states.head(), x='DEPARTAMENTO', y='CANTIDAD', title='Top 5 of Colombian states most impacted by thefts levels')
fig.show()

In [31]:
#checking which 'municipios' (cities) are more prone to theft and robbery
df_pivot_cities = df.groupby(["MUNICIPIO"])["CANTIDAD"].sum().reset_index().sort_values(by="CANTIDAD", ascending=False)
df_pivot_cities.MUNICIPIO = df_pivot_cities.MUNICIPIO.str.rstrip(' (CT)')
df_pivot_cities.head(10)

,MUNICIPIO,CANTIDAD
99,BOGOTÁ D.C.,559762
129,CALI,121469
509,MEDELLÍN,118113
83,BARRANQUILLA,67060
106,BUCARAMANGA,41009
994,VILLAVICENCIO,34366
606,PASTO,30291
397,IBAGUÉ,29475
151,CARTAGENA,27716
553,NEIVA,23552


In [32]:
#Graph for top 5 of cities in Colombia most impacted by thefts
fig = px.bar(df_pivot_cities.head(), x='MUNICIPIO', y='CANTIDAD', title='Top 5 of Colombian cities most impacted by thefts levels')
fig.show()

#### Top 5 areas impacted by theft finding:

The most affected states are driven by its main capital cities crimes as can be seen. Most thefts / robbery distribution is focused on cities instead of towns or counties. 

Top 5 most impacted cities in Colombia by thefts are:

1. Bogota
2. Cali
3. Medellin
4. Barranquilla
5. Bucaramanga

Cali is 3rd biggest city after Medellin, even though managed to get in top 2 of theft levels.

These data is based on 2018 census from DANE which is the latest official stat about population. DANE is the institution in charge of handling statistics in Colombia.

In [80]:
#checking which 'departamentos' (states) have more unsatisfied basic needs or poverty
ubn_states = ubn_states.reset_index().sort_values(by="Prop de Personas en NBI (%)", ascending=False)
ubn_states = ubn_states.drop(columns=['Componente vivienda', 'Componente Servicios','Componente Hacinamiento',
                         'Componente Inasistencia', 'Componente dependencia económica', 'Código Departamento', 'index'])
ubn_states.head(10)

,Nombre Departamento,Prop de Personas en NBI (%),Prop de Personas en miseria
31,VAUPÉS,68.887891,43.341505
32,VICHADA,67.622715,50.675723
11,CHOCÓ,65.395425,20.223436
29,GUAINÍA,59.206535,35.304135
13,LA GUAJIRA,53.010826,30.220680
28,AMAZONAS,34.915914,10.809029
9,CÓRDOBA,34.819303,11.161460
24,ARAUCA,32.264537,9.294772
21,SUCRE,28.981253,7.938032
30,GUAVIARE,27.799245,8.969520


In [81]:
#checking which 'municipios' (cities and towns) have more unsatisfied basic needs or poverty
ubn_cities = ubn_cities.reset_index().sort_values(by="Prop de Personas en NBI (%)", ascending=False)
ubn_cities = ubn_cities.drop(columns=['Componente vivienda', 'Componente Servicios','Componente Hacinamiento',
                         'Componente Inasistencia', 'Componente dependencia económica', 'Código Departamento', 
                         'index', 'Código Municipio'])
ubn_cities.head(10)

,Nombre Departamento,Nombre Municipio,Prop de Personas en NBI (%),Prop de Personas en miseria
1103,GUAINÍA,PUERTO COLOMBIA (ANM),95.963756,69.357496
1114,VAUPÉS,PACOA (ANM),93.647469,82.024433
1104,GUAINÍA,LA GUADALUPE (ANM),93.630573,46.496815
179,BOLÍVAR,SAN JACINTO,93.414292,29.591100
1107,GUAINÍA,MORICHAL (ANM),92.366412,69.618321
1106,GUAINÍA,PANA PANA (ANM),91.640867,63.364293
1092,AMAZONAS,LA VICTORIA (ANM),90.963855,66.265060
1102,GUAINÍA,SAN FELIPE (ANM),88.391225,58.409506
653,LA GUAJIRA,URIBIA,88.256010,60.349253
1121,VICHADA,CUMARIBO,87.370476,73.552085


#### Unsatisfied Basic Needs (UBN) finding:

This data is taken from DANE census in 2018, most recent stat about poverty in Colombia. NBI is UBN in spanish.

Since this data is a snapshot, not a time series like the rest of data, we can not make correlations; but at simple sight from 32 states (Departamentos) none of top 10 states in UBN are not in none of top 10 most affected states by thefts.

This finding might suggest that generallized poverty is not a driver to commit this crime. By the way, these high UBN areas are far away from main cities where crime levels are high. 

Also, in cities, compared to towns, there are much less levels of UBN but still there are poor and rich neighborhoods. There are no official data as snapshot nor as time series for digging further in order to confirm or reject hypothesis of inequality being a driver for thefts.

###Theft rate data

States theft's levels main drivers are its capital cities as can be seen on plots above. Meaning that small towns are not big contributors to these crime levels.

These analyses and plots are for population adjusted data with thefts levels per 100,000 habitants to ease comparison but bearing in mind that it is useful just for cities with more than 500,000 habitants in order to avoid small towns affect stats.

This rate data is the official indicator for comparison purposes against internal and external / foreign cities across the world.

In [35]:
#merging 2 dataframes for adjusting rate per 100,000 habitants in order to eliminate population noise from raw data
cities_data = pd.merge(df_pivot_cities, pop_cities, left_on='MUNICIPIO', right_on='MUNICIPIO', how='inner' )
cities_data['rate'] = round((cities_data['CANTIDAD']/cities_data['TOTAL'])*100000, 2)
cities_data.sort_values(by='rate', ascending=False, inplace=True)
#filter cities with more than 500,000 habitants in order to avoid false positives rates due to small towns included in MUNICIPIOS list
cities_data = cities_data[cities_data['TOTAL']>500000]

cities_data.head(10)

,MUNICIPIO,CANTIDAD,DEPARTAMENTO,TOTAL,rate
0,BOGOTÁ D.C.,559762,CUNDINAMARCA,7412566,7551.53
4,BUCARAMANGA,41009,SANTANDER,581130,7056.77
5,VILLAVICENCIO,34366,META,531275,6468.59
7,IBAGUÉ,29475,TOLIMA,529635,5565.15
3,BARRANQUILLA,67060,ATLÁNTICO,1206319,5559.06
1,CALI,121469,VALLE DEL CAUCA,2227642,5452.81
2,MEDELLÍN,118113,ANTIOQUIA,2427129,4866.37
8,CARTAGENA,27716,BOLÍVAR,973045,2848.38
14,SOACHA,17943,CUNDINAMARCA,660179,2717.90
11,CÚCUTA,19217,NORTE DE SANTANDER,711715,2700.10


In [36]:
#Graph for top 5 of states in Colombia most impacted by thefts
fig = px.bar(cities_data.head(), x='MUNICIPIO', y='rate', title='Top 5 of Colombian cities with higher rate of thefts levels per 100,000 hab')
fig.show()

#### Theft rate finding:

Bogota is still number 1 in both top 5's, rated and unrated theft levels.

Barranquilla, being 4th biggest city in population is in both top 5's.

Bucaramanga, which has around half of Barranquilla's population has a higher theft rate. Bucaramanga, Villavicencio and Ibague are considered mid size cities in Colombia and are 9th, 10th, 11th in population size as per latest census data in 2018.

In [37]:
#merging 2 dataframes for adjusting rate per 100,000 habitants in order to eliminate population noise from raw data
states_data = pd.merge(df_pivot_states, pop_states, left_on='DEPARTAMENTO', right_on='DEPARTAMENTO', how='inner' )
states_data['rate'] = round((states_data['CANTIDAD']/states_data['TOTAL'])*100000, 2)
states_data.sort_values(by='rate', ascending=False, inplace=True)
#filter cities with more than 500,000 habitants in order to avoid false positives rates due to small states included in DEPARTAMENTOS list
states_data = states_data[states_data['TOTAL']>500000]

states_data.head(10)

,DEPARTAMENTO,CANTIDAD,TOTAL,rate
0,CUNDINAMARCA,621378,10331626,6014.33
6,META,40924,1039722,3936.05
2,VALLE,168055,4475886,3754.68
4,SANTANDER,80744,2184837,3695.65
3,ATLÁNTICO,85586,2535517,3375.49
5,TOLIMA,44181,1330187,3321.41
10,RISARALDA,30264,943401,3207.97
9,HUILA,34283,1100386,3115.54
1,ANTIOQUIA,174183,6407102,2718.59
12,CALDAS,25434,998255,2547.85


In [38]:
#Graph for top 5 of states in Colombia most impacted by thefts
fig = px.bar(states_data.head(), x='DEPARTAMENTO', y='rate', title='Top 5 of Colombian states with higher rate of thefts levels per 100,000 hab')
fig.show()

### Arrests data

In [39]:
#change format to DATE column in order to convert later to datetime format
arrests["fecha"] = pd.to_datetime(arrests["FECHA"]).dt.strftime('%Y/%m/%d')

# To convert a string to a datetime
arrests["DateTime"] = pd.to_datetime(arrests["fecha"], format="%Y/%m/%d")

# Set index as DateTime for plotting purposes
arrests = arrests.set_index(["DateTime"])

# To get month and year column
arrests["month"] = pd.DatetimeIndex(arrests["fecha"]).month
arrests["year"] = pd.DatetimeIndex(arrests["fecha"]).year
arrests['month_year'] = pd.to_datetime(arrests["fecha"]).dt.to_period('M')
arrests["day"] = pd.to_datetime(arrests["fecha"]).dt.weekday

arrests.head()

,DEPARTAMENTO,MUNICIPIO,CODIGO DANE,FECHA,GENERO,DELITO,EDAD,CANTIDAD,fecha,month,year,month_year,day
DateTime,,,,,,,,,,,,,
2011-01-01,AMAZONAS,LETICIA (CT),91001000.0,2011-01-01,MASCULINO,ARTÍCULO 189. VIOLACIÓN DE HABITACIÓN AJENA.,ADULTOS,1,2011/01/01,1,2011,2011-01,5
2011-01-01,AMAZONAS,LETICIA (CT),91001000.0,2011-01-01,MASCULINO,ARTÍCULO 111. LESIONES PERSONALES,ADULTOS,1,2011/01/01,1,2011,2011-01,5
2011-01-03,AMAZONAS,LETICIA (CT),91001000.0,2011-01-03,MASCULINO,ARTÍCULO 296. FALSEDAD PERSONAL,ADULTOS,1,2011/01/03,1,2011,2011-01,0
2011-01-04,AMAZONAS,LETICIA (CT),91001000.0,2011-01-04,FEMENINO,ARTÍCULO 111. LESIONES PERSONALES,ADULTOS,1,2011/01/04,1,2011,2011-01,1
2011-01-05,AMAZONAS,LETICIA (CT),91001000.0,2011-01-05,MASCULINO,"ARTÍCULO 376. TRÁFICO, FABRICACIÓN O PORTE DE ...",ADULTOS,1,2011/01/05,1,2011,2011-01,2


In [84]:
#Creating a pivot table for each year to get a close up on values per year
arrests_pivot_year = arrests.groupby(["DELITO"])["CANTIDAD"].sum().reset_index().sort_values(by="CANTIDAD", ascending=False).head()
fig = px.bar(arrests_pivot_year, x='DELITO', y='CANTIDAD', title='Top 10 arrests per crimes in Colombia - Jan 01st 2010 upto Nov 30th 2020')
fig.show()

In [41]:
#Creating a pivot table for each year to get a close up on values per year
arrests_pivot_year = arrests.groupby(["year"])["CANTIDAD"].sum().reset_index().sort_values(by="year")
px.line(x="year" , y="CANTIDAD", data_frame=arrests_pivot_year, title="Arrests per Year in Colombia due to multiple crimes - Jan 01st 2010 upto Nov 30th 2020")

In [42]:
#Creating a pivot table for each year to get a close up on values per year of arrests per crime type
arrests_pivot_year2 = arrests.groupby(["year", "DELITO"])["CANTIDAD"].sum().reset_index().sort_values(by="year")
px.line(x="year" , y="CANTIDAD", line_group="DELITO", data_frame=arrests_pivot_year2, title="Arrests per crime and year in Colombia - Jan 01st 2010 upto Nov 30th 2020")

#### Arrest per crime finding:

Colombia being one of main producers of illicit drugs worldwide, its police is focused on this crime and the war on drugs (ARTÍCULO 376. TRÁFICO, FABRICACIÓN O PORTE DE ESTUPEFACIENTES). Thus, no surprise.

Second most important arrest per crime category is theft (ARTICULO 239, HURTO PERSONAS). This helps to explain the importance of this analysis.

In [43]:
arrests_pivot_year3 = arrests_pivot_year2[arrests_pivot_year2['DELITO']=='ARTÍCULO 239. HURTO PERSONAS']
arrests_pivot_year3.rename(columns={'CANTIDAD':'TOTAL'}, inplace=True)
arrests_pivot_year3

,year,DELITO,TOTAL
99,2011,ARTÍCULO 239. HURTO PERSONAS,34969
373,2012,ARTÍCULO 239. HURTO PERSONAS,40290
653,2013,ARTÍCULO 239. HURTO PERSONAS,41829
932,2014,ARTÍCULO 239. HURTO PERSONAS,42454
1201,2015,ARTÍCULO 239. HURTO PERSONAS,38724
1461,2016,ARTÍCULO 239. HURTO PERSONAS,38156
1715,2017,ARTÍCULO 239. HURTO PERSONAS,35093
1972,2018,ARTÍCULO 239. HURTO PERSONAS,32296
2235,2019,ARTÍCULO 239. HURTO PERSONAS,30508
2482,2020,ARTÍCULO 239. HURTO PERSONAS,19249


In [44]:
#Creating a pivot table for each year to get a close up on values per year of arrests per crime type
px.line(x="year" , y="TOTAL", data_frame=arrests_pivot_year3, title="Arrests per year in Colombia due to theft / robbery - Jan 01st 2010 upto Nov 30th 2020")

#### Arrests due to theft finding:

There is a downward trend in arrests due to theft category since 2014. 

2020 is an atypical year due to lockdown but previous trend is worrisome.

In [45]:
#merging 2 dataframes for getting % of arrest per theft crime under analysis
eff = pd.merge(df_pivot_year, arrests_pivot_year3, left_on='year', right_on='year', how='inner' )
eff['%arrests'] = round(eff['TOTAL']/eff['CANTIDAD']*100, 2)
eff

,year,CANTIDAD,DELITO,TOTAL,%arrests
0,2011,67866,ARTÍCULO 239. HURTO PERSONAS,34969,51.53
1,2012,82656,ARTÍCULO 239. HURTO PERSONAS,40290,48.74
2,2013,93455,ARTÍCULO 239. HURTO PERSONAS,41829,44.76
3,2014,94427,ARTÍCULO 239. HURTO PERSONAS,42454,44.96
4,2015,101449,ARTÍCULO 239. HURTO PERSONAS,38724,38.17
5,2016,146866,ARTÍCULO 239. HURTO PERSONAS,38156,25.98
6,2017,209784,ARTÍCULO 239. HURTO PERSONAS,35093,16.73
7,2018,257072,ARTÍCULO 239. HURTO PERSONAS,32296,12.56
8,2019,306847,ARTÍCULO 239. HURTO PERSONAS,30508,9.94
9,2020,183240,ARTÍCULO 239. HURTO PERSONAS,19249,10.50


In [85]:
#df.describe()

In [47]:
#Creating a pivot table for each year to get a close up on values per year of arrests per crime type
px.line(x="year" , y="%arrests", data_frame=eff, title="% Arrests per year in Colombia on theft / robbery - Jan 01st 2010 upto Nov 30th 2020")

#### % efficiency of theft arrest finding:

Efficiency of police regarding theft category is decreasing since 2011. Interesting finding that might help to support why security perception within citizens is worsening.

### Prison data

In [48]:
#Plotting recidivism level through period of analysis with data available from INPEC
px.line(x="Periodo" , y="Reincidencia", data_frame=rcdv, title="Recidivism in Colombia - Jan 01st 2010 upto Nov 30th 2020")

#### Recidivism finding:

There is an upward trend in recidivism since 2016 upto 2020 bearing in mind that in 2020 arrests decreased due to lockdown.

Interesting finding, as time goes by, felons are more inclided to re-offend after being released of prison.

In [49]:
#Plotting detention months of unconvicted inmates through period of analysis with data available from INPEC
fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=dmu['Periodo'], y=dmu['0-5'], name='0-5 months'))
fig.add_trace(go.Scatter(x=dmu['Periodo'], y=dmu['6-10'], name='6-10 months'))
fig.add_trace(go.Scatter(x=dmu['Periodo'], y=dmu['11-15'], name = '11-15 months'))
fig.add_trace(go.Scatter(x=dmu['Periodo'], y=dmu['16-20'],name='16-20 months'))
fig.add_trace(go.Scatter(x=dmu['Periodo'], y=dmu['21-25'], name='21-25 months'))
fig.add_trace(go.Scatter(x=dmu['Periodo'], y=dmu['26-30'], name='26-30 months'))
fig.add_trace(go.Scatter(x=dmu['Periodo'], y=dmu['31-35'], name='31-35 months'))
fig.add_trace(go.Scatter(x=dmu['Periodo'], y=dmu['>36'], name='>36 months'))

# Edit the layout
title_string = 'Time Series per year of detention months of unconvicted inmates in Colombia'
fig.update_layout(title=title_string, xaxis_title='Year', yaxis_title='# Population')
fig.update_traces(mode='lines')
fig.show()

#### Detention months of unconvicted inmates finding:

A sharp decrease in detention months of unconvicted inmates since 2018 within range of 0-5 and 6-10 months with no increase in other time range. 

This finding means a higher turn over in prison systems or an increase in speed of judicial systems for inmate conviction process.

In [58]:
#Plotting detention years of convicted inmates through period of analysis with data available from INPEC
fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=dyc['Periodo'], y=dyc['0-5'], name='0-5 years'))
fig.add_trace(go.Scatter(x=dyc['Periodo'], y=dyc['6-10'], name='6-10 years'))
fig.add_trace(go.Scatter(x=dyc['Periodo'], y=dyc['11-15'], name = '11-15 years'))
fig.add_trace(go.Scatter(x=dyc['Periodo'], y=dyc['16-20'],name='16-20 years'))
fig.add_trace(go.Scatter(x=dyc['Periodo'], y=dyc['21-25'], name='21-25 years'))
fig.add_trace(go.Scatter(x=dyc['Periodo'], y=dyc['26-30'], name='26-30 years'))
fig.add_trace(go.Scatter(x=dyc['Periodo'], y=dyc['31-35'], name='31-35 years'))
fig.add_trace(go.Scatter(x=dyc['Periodo'], y=dyc['>36'], name='>36 years'))

# Edit the layout
title_string = 'Time Series of detention years of convicted inmates in Colombia'
fig.update_layout(title=title_string, xaxis_title='Year', yaxis_title='# Population')
fig.update_traces(mode='lines')
fig.show()

#### Detention years of convited inmates finding:

A similar behaviour as detention month for unconvited inmates a decrease in range of 0-5 and 6-10 years with no increase in other time ranges.

This finding means a higher turn over in prison systems. 

In [51]:
#Plotting prison capacity vs prison population through period of analysis with data available from INPEC
fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=tpp['Periodo'], y=tpp['Capacidad'], name='Capacity'))
fig.add_trace(go.Scatter(x=tpp['Periodo'], y=tpp['Poblacion'], name='Population'))
fig.add_trace(go.Scatter(x=tpp['Periodo'], y=tpp['Sindicados'], name = 'Unconvicted'))
fig.add_trace(go.Scatter(x=tpp['Periodo'], y=tpp['Condenados'],name='Convicted'))

# Edit the layout
title_string = 'Time Series of Capacity vs Population of inmates in Colombia'
fig.update_layout(title=title_string, xaxis_title='Year', yaxis_title='# Population')
fig.update_traces(mode='lines')
fig.show()

#### Prison's Capacity vs Population finding:

There is a gap between capacity and population meaning overcrowding in prisons. 

During last period from 2019 to 2020 there is a decrease of prison's population due to previously analysed decreases in unconvited and convicted population within Colombian jails.

Capacity is almost flat with very small changes due to the fact that it is expensive to build new jails with latest security standards in an undeveloped country with other more urgent needs in its society and therefore with budget constraints.

In [52]:
#Calculate overcrowding for plotting and understand trend through period of analysis
tpp['% Overcrowding'] = round((tpp['Poblacion']/tpp['Capacidad']*100)-100, 2)
#tpp['% Overcrowding']
#Plotting % overcrowding through period of analysis with data available from INPEC
px.line(x="Periodo" , y="% Overcrowding", data_frame=tpp, title="% Overcrowding in prisons of Colombia - Jan 01st 2010 upto Nov 30th 2020")

#### % Overcrowding finding:

% overcrowding since 2014 upto 2019 was around 45% and 55% above capacity but in 2020 dropped to 20%. 

It is either a big coincidence or an attempt to reduce overcrowding in prison systems from INPEC during coronavirus lockdown. In 2020 there records of riots in Colombian prisons in press media. 

## Top 5 cities most impacted by theft analysis

In [96]:
#Creating a plot for thefts and arrests only due to thefts in city under analysis
df_bta = df[df['MUNICIPIO']=='BOGOTÁ D.C. (CT)'].groupby(["year"])["CANTIDAD"].sum().reset_index().sort_values(by="year")
df_bta2 = arrests[arrests['MUNICIPIO']=='BOGOTÁ D.C. (CT)']
df_bta2 = df_bta2[df_bta2['DELITO']=='ARTÍCULO 239. HURTO PERSONAS']
df_bta2 = df_bta2.groupby(["year"])["CANTIDAD"].sum().reset_index().sort_values(by="year")

fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=df_bta['year'], y=df_bta['CANTIDAD'], name='Thefts'))
fig.add_trace(go.Scatter(x=df_bta2['year'], y=df_bta2['CANTIDAD'], name='Arrests'))

# Edit the layout
title_string = 'Time Series of Thefts vs Arrests in Bogota'
fig.update_layout(title=title_string, xaxis_title='Year', yaxis_title='# Population')
fig.update_traces(mode='lines')
fig.show()

In [97]:
#Creating a plot for thefts and arrests only due to thefts in city under analysis
df_cali = df[df['MUNICIPIO']=='CALI (CT)'].groupby(["year"])["CANTIDAD"].sum().reset_index().sort_values(by="year")
df_cali2 = arrests[arrests['MUNICIPIO']=='CALI (CT)']
df_cali2 = df_cali2[df_cali2['DELITO']=='ARTÍCULO 239. HURTO PERSONAS']
df_cali2 = df_cali2.groupby(["year"])["CANTIDAD"].sum().reset_index().sort_values(by="year")

fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=df_cali['year'], y=df_cali['CANTIDAD'], name='Thefts'))
fig.add_trace(go.Scatter(x=df_cali2['year'], y=df_cali2['CANTIDAD'], name='Arrests'))

# Edit the layout
title_string = 'Time Series of Thefts vs Arrests in Cali'
fig.update_layout(title=title_string, xaxis_title='Year', yaxis_title='# Population')
fig.update_traces(mode='lines')
fig.show()

In [98]:
#Creating a plot for thefts and arrests only due to thefts in city under analysis
df_mdl = df[df['MUNICIPIO']=='MEDELLÍN (CT)'].groupby(["year"])["CANTIDAD"].sum().reset_index().sort_values(by="year")
df_mdl2 = arrests[arrests['MUNICIPIO']=='MEDELLÍN (CT)']
df_mdl2 = df_mdl2[df_mdl2['DELITO']=='ARTÍCULO 239. HURTO PERSONAS']
df_mdl2 = df_mdl2.groupby(["year"])["CANTIDAD"].sum().reset_index().sort_values(by="year")

fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=df_mdl['year'], y=df_mdl['CANTIDAD'], name='Thefts'))
fig.add_trace(go.Scatter(x=df_mdl2['year'], y=df_mdl2['CANTIDAD'], name='Arrests'))

# Edit the layout
title_string = 'Time Series of Thefts vs Arrests in Medellin'
fig.update_layout(title=title_string, xaxis_title='Year', yaxis_title='# Population')
fig.update_traces(mode='lines')
fig.show()

In [99]:
#Creating a plot for thefts and arrests only due to thefts in city under analysis
df_bqa = df[df['MUNICIPIO']=='BARRANQUILLA (CT)'].groupby(["year"])["CANTIDAD"].sum().reset_index().sort_values(by="year")
df_bqa2 = arrests[arrests['MUNICIPIO']=='BARRANQUILLA (CT)']
df_bqa2 = df_bqa2[df_bqa2['DELITO']=='ARTÍCULO 239. HURTO PERSONAS']
df_bqa2 = df_bqa2.groupby(["year"])["CANTIDAD"].sum().reset_index().sort_values(by="year")

fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=df_bqa['year'], y=df_bqa['CANTIDAD'], name='Thefts'))
fig.add_trace(go.Scatter(x=df_bqa2['year'], y=df_bqa2['CANTIDAD'], name='Arrests'))

# Edit the layout
title_string = 'Time Series of Thefts vs Arrests in Barranquilla'
fig.update_layout(title=title_string, xaxis_title='Year', yaxis_title='# Population')
fig.update_traces(mode='lines')
fig.show()

In [100]:
#Creating a plot for thefts and arrests only due to thefts in city under analysis
df_bga = df[df['MUNICIPIO']=='BUCARAMANGA (CT)'].groupby(["year"])["CANTIDAD"].sum().reset_index().sort_values(by="year")
df_bga2 = arrests[arrests['MUNICIPIO']=='BUCARAMANGA (CT)']
df_bga2 = df_bga2[df_bga2['DELITO']=='ARTÍCULO 239. HURTO PERSONAS']
df_bga2 = df_bga2.groupby(["year"])["CANTIDAD"].sum().reset_index().sort_values(by="year")

fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=df_bga['year'], y=df_bga['CANTIDAD'], name='Thefts'))
fig.add_trace(go.Scatter(x=df_bga2['year'], y=df_bga2['CANTIDAD'], name='Arrests'))

# Edit the layout
title_string = 'Time Series of Thefts vs Arrests in Bucaramanga'
fig.update_layout(title=title_string, xaxis_title='Year', yaxis_title='# Population')
fig.update_traces(mode='lines')
fig.show()


#### Top 5 cities most affected by theft findings:

In all 5 cases, the gap between thefts cases and arrest cases due to thefts are widening, meaning an increase in security issues for citizens of these 5 cities.

## Conclusions:

1. There is an upward trend in thefts at country level since 2010 upto 2019; but there is an increased rate in theft levels after 2015. 2020 was an atypical year due to lockdown.

2. Almost 50% of thefts are made without weapons (non violent thefts), the remaining 50% are violent with risk of harm or life threat.

3. The most affected states are driven by its main capital cities crimes as can be seen. Most thefts / robbery distribution is focused on cities (urban areas) instead of towns or counties (rural areas). This answers question # 1 partially.

    3.1. Top 5 most impacted cities in Colombia by thefts are:

    Bogota
    Cali
    Medellin
    Barranquilla
    Bucaramanga

4. Bogota is still number 1 in both top 5's, rated and unrated theft levels. Barranquilla, being 4th biggest city in population is in both top 5's. Bucaramanga, which has around half of Barranquilla's population has a higher theft rate. This finding answers question # 2 which asks about how to compare rate among internal and external / foreign cities.

5. Since UBN data is a snapshot, not a time series like the rest of data, we can not make correlations; but at simple sight from 32 states (Departamentos) none of top 10 states in UBN are not in none of top 10 most affected states by thefts. This finding might suggest that generallized poverty is not a driver to commit this crime. By the way, these high UBN areas are far away from main cities where crime levels are high. Also, in cities, compared to towns, there are much less levels of UBN but still there are poor and rich neighborhoods. There are no official data as snapshot nor as time series for digging further in order to confirm or reject hypothesis of inequality being a driver for thefts. Therefore, question # 1 can not be answered completely due to lack of data availabity on unequality within Colombian cities.

6. Colombia being one of main producers of illicit drugs worldwide, its police is focused on the war on drugs (ARTÍCULO 376. TRÁFICO, FABRICACIÓN O PORTE DE ESTUPEFACIENTES). Second most important arrest per crime category is theft (ARTICULO 239, HURTO PERSONAS). This helps to explain the importance of this analysis.

7. There is a downward trend in arrests due to theft category since 2014 this trend is worrisome. Conclusions # 1 and # 7 answers question # 3.

8. Efficiency of police regarding theft category is a decreasing trend  since 2011. Interesting finding that might help to support why security perception within citizens is worsening. Police efforts are focused on war on drugs and in a lesser way to reduce thefts as can be seen on plots based on official records. This finding answers question # 4.

9. There is an upward trend in recidivism since 2016 upto 2020. Interesting finding, as time goes by, felons are more inclided to re-offend after being released of prison. This finding answers partially question # 5.

10. A sharp decrease in detention months of unconvicted inmates since 2018 within range of 0-5 and 6-10 months, this finding means a higher turn over in prison systems or an increase in speed of judicial systems for inmate conviction process. 

11. A similar behaviour as detention months for unconvited inmates a decrease in range of 0-5 and 6-10 years of convited inmates, this finding means a higher turn over in prison systems. This also helps to explains why citizens are concerned with security perception worsening on streets.

12. There is a gap between capacity and population meaning overcrowding in prisons. During last period from 2019 to 2020 there is a decrease of prison's population due to previously analysed decreases in unconvited and convicted population within Colombian jails. Capacity is almost flat with very small changes due to the fact that it is expensive to build new jails with latest security standards in an undeveloped country with other more urgent needs in its society and therefore with budget constraints to try to solve this overcrowding issue. 

13. % overcrowding since 2014 upto 2019 was around 45% and 55% above capacity but in 2020 dropped to 20%. It is either a big coincidence or an attempt to reduce overcrowding in prison systems from INPEC during coronavirus lockdown. In 2020 there are records of riots in Colombian prisons on press media. This finding answers partially question # 5 because there is correlation between recidivism and overcrowding documented on multiple studies: [Universidad EAFIT](https://poseidon01.ssrn.com/delivery.php?ID=797101088103070016103114090083084095061040041017050027018015064117008114007024116112101013061121056036119094117095019084068065043013049092040071082092021004088007005060044012072115081118027123030013011115092025123001094069067124108117102089027023072009&EXT=pdf&INDEX=TRUE), [University of Chicago](https://www.jstor.org/stable/1147497?seq=1), and [University of Geneve](https://www.frontiersin.org/articles/10.3389/fpsyt.2019.01015/full).

14. In all of the top 5 cases, the gap between thefts cases and arrest cases due to thefts are widening, meaning an increase in security issues for citizens of these 5 cities. This also helps to explains why citizens are concerned with security perception worsening on cities' streets. This confirms answers to question # 3 given on conclusions # 1 and 7.